# Character-Level Tokenization and Embeddings

This notebook demonstrates character-level tokenization and embeddings using the Shakespeare dataset.

In [1]:
import torch
import numpy as np
import rich

from pathlib import Path
import os
import requests

In [2]:
# Download sample dataset 
data_dir = Path("data")
data_dir.mkdir(exist_ok=True)

# URL for the Shakespeare dataset
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"

# Download the file
print("Downloading Shakespeare dataset...")
response = requests.get(url)

if response.status_code == 200:
    # Save the file
    output_path = data_dir / "shakespeare.txt"
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(response.text)
    print(f"Dataset downloaded successfully to {output_path}")
    print(f"File size: {len(response.text)} characters")
else:
    print(f"Error downloading file. Status code: {response.status_code}")

Dataset downloaded successfully to data/shakespeare.txt
File size: 1115394 characters


## Load and Inspect the Data

In [3]:
# 40,000 lines of Shakespeare from a variety of Shakespeare's plays
with open('data/shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(f'Length of dataset in characters: {len(text)}')
print('\nFirst 1000 characters:\n')
print(text[:1000])

Length of dataset in characters: 1115394

First 1000 characters:

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods k

## Character-Level Tokenization

In character-level tokenization, each unique character in the text becomes a token. This is the simplest form of tokenization.

In [ ]:
# Get all unique characters in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)

print(f'Vocabulary size (unique characters): {vocab_size}')
print('\nAll characters:', ''.join(chars))

In [ ]:
# Create mappings from characters to integers and back
char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for i, ch in enumerate(chars)}

# Example: encode and decode some text
example_text = "Hello, World!"
encoded = [char_to_idx[ch] for ch in example_text]
decoded = ''.join([idx_to_char[idx] for idx in encoded])

print(char_to_idx)
print(f'Original text: {example_text}')
print(f'Encoded: {encoded}')
print(f'Decoded: {decoded}')

## Character Embeddings

Now we'll create character embeddings. Each character will be represented by a vector in a higher-dimensional space.

In [ ]:
# Parameters for embeddings
embedding_dim = 2  # Dimension of the embedding space

# Create a random embedding table
embeddings = torch.nn.Embedding(vocab_size, embedding_dim)

# Example: get embeddings for a sequence of characters
example_sequence = torch.tensor(encoded)  # Using the encoded text from above
embedded_sequence = embeddings(example_sequence)

print(f'Shape of embedded sequence: {embedded_sequence.shape}')
print('\nEmbedding for first character:')
print(embedded_sequence[0].detach().numpy())

In [ ]:
encoded

In [ ]:
embeddings.weight.detach().numpy()

## Visualizing Embeddings

We can visualize the learned embeddings by projecting them to 2D using PCA. As it's only 2D, we can directly visualise them.

In [ ]:
import matplotlib.pyplot as plt

# Get the embedding weights
weights = embeddings.weight.detach().numpy()

# Plot
plt.figure(figsize=(12, 8))
plt.scatter(weights[:, 0], weights[:, 1], alpha=0.5)

# Add character labels
for i, char in enumerate(chars):
    if char == '\n': char = 'newline'  # Make newline visible
    plt.annotate(char, (weights[i, 0], weights[i, 1]))

plt.title('Character Embeddings Projected to 2D')
plt.xlabel('First Component')
plt.ylabel('Second Component')
plt.grid(True)
plt.show()

# Train Embedding function

In [ ]:
# Example of how we create training pairs
def show_data_preparation_example():
    # Take a small sample of text
    sample_text = "Hello, World!"
    print(f"Sample text: {sample_text}")
    
    # Convert to indices
    indices = [char_to_idx[ch] for ch in sample_text]
    context_size = 2
    
    print("\nCreating context windows (context_size=2):")
    print("Center char | Context chars (2 before, 2 after)")
    print("-" * 45)
    
    # Show some examples
    for i in range(context_size, len(indices) - context_size):
        center_char = sample_text[i]
        context_chars = (
            sample_text[i-context_size:i] + '*' +
            sample_text[i+1:i+context_size+1]
        )
        
        print(f"    {center_char}     | {context_chars}")
        
    print("\nThese will be converted to tensor indices for training.")
    
# Run the example
show_data_preparation_example()

In [ ]:
# Convert text to indices tensor
data = torch.tensor([char_to_idx[ch] for ch in text], dtype=torch.long)

print(f"Shape of data tensor: {data.shape}")
print(f"First 20 characters as indices: {data[:20]}")
print(f"As characters: {''.join([idx_to_char[idx.item()] for idx in data[:20]])}")

In [ ]:
# Model parameters
embedding_dim = 2
context_size  = 2
batch_size    = 1024
learning_rate = 0.001

In [ ]:
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np

class SkipgramDataset(Dataset):
    def __init__(self, data, context_size):
        self.data = data
        self.context_size = context_size
        self.pairs = []
        
        # Create training pairs
        for i in range(context_size, len(data) - context_size):
            center = data[i]
            context = torch.cat([
                data[i-context_size:i],
                data[i+1:i+context_size+1]
            ])
            self.pairs.append((center, context))
    
    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self, idx):
        center, context = self.pairs[idx]
        return center, context

# Create dataset and dataloader
dataset = SkipgramDataset(data, context_size)

train_size = int(0.9*len(dataset))
val_size   = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_dataloader           = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_dataloader      = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

rich.print(f'Number of training examples: {len(train_dataset):,}')
rich.print(f'Number of validation examples: {len(val_dataset):,}')
rich.print(f'Number of batches: {len(train_dataloader):,}')

In [ ]:
datapoint = 0
center    = idx_to_char[dataset[datapoint][0].item()]
context   = ''
for val in dataset[datapoint][1]:
    context += idx_to_char[val.item()]

print( f'Center: {center} | Context: {context[:2]}*{context[2:]}' )

# 1. Simplest approach

In [ ]:
import torch
import torch.nn as nn

class CharacterEmbedding(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        # Embedding layer for context characters
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        # Output layer to predict center character
        self.output = nn.Linear(embedding_dim, vocab_size)
        
    def forward(self, center_chars, context_chars):
        # Get embeddings for context characters
        context_embeds = self.embeddings(context_chars)  # [batch_size, context_size*2, embedding_dim]
        
        # Average context embeddings
        context_embeds = context_embeds.mean(dim=1)  # [batch_size, embedding_dim]
        
        # Predict center character
        logits = self.output(context_embeds)  # [batch_size, vocab_size]
        
        return logits

In [ ]:
# Initialize model, optimizer, and loss function
model = CharacterEmbedding(vocab_size, embedding_dim)

# Use AdamW optimizer with weight decay for better regularization
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Keep using CrossEntropyLoss
criterion = nn.CrossEntropyLoss()

print(model)

In [ ]:
def train_embedding_epoch( model, train_dataloader, val_dataloader, optimizer, criterion):
    
    # -- Training loop --
    model.train()
    train_loss = 0
    with tqdm(train_dataloader, desc=f'Epoch {epoch+1}/{n_epochs}') as pbar:
        for center, context in pbar:
            # Forward pass
            logits = model(center, context)
            
            # Compute loss
            loss = criterion(logits, center)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # Update metrics
            train_loss += loss.item()
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})
            
    train_loss = train_loss / len(train_dataloader)

    # --- Validation loop ---
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for center, context in val_dataloader:
            logits    = model(center, context)
            loss      = criterion(logits, center)
            val_loss += loss.item()
            
    validation_loss = val_loss / len(val_dataloader)

    return train_loss, validation_loss

def plot_losses( losses, val_losses ):
    clear_output(wait=True)
    plt.figure(figsize=(10, 5))
    plt.plot(losses, label='Train')
    plt.plot(val_losses, label='Val')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss Over Epochs')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from IPython.display import clear_output

# Training loop
n_epochs = 20
losses, val_losses = [], []

for epoch in range(n_epochs):
    train_loss, val_loss = train_embedding_epoch( model, train_dataloader, validation_dataloader, optimizer, criterion )

    losses.append( train_loss )
    val_losses.append( val_loss )
    
    # Plot the loss
    plot_losses( losses, val_losses )

In [ ]:
# Get the trained embeddings (we use the main embeddings, not the context ones)
trained_embeddings = model.embeddings.weight.detach().numpy()

# Plot
plt.figure(figsize=(15, 10))
plt.scatter(trained_embeddings[:, 0], trained_embeddings[:, 1], alpha=0.5)

# Add character labels
for i, char in enumerate(chars):
    if char == '\n': char = 'newline'  # Make newline visible
    plt.annotate(char, (trained_embeddings[i, 0], trained_embeddings[i, 1]))

plt.title('Character Embeddings 2D')
plt.xlabel('First Component')
plt.ylabel('Second Component')
plt.grid(True)
plt.show()

# 2. Add a hidden layer 

In [ ]:
class CharacterEmbedding(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim=64):
        super().__init__()
        
        # Embedding layer
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)

        # Add batch normalization for better training stability
        self.batch_norm = nn.BatchNorm1d(embedding_dim)
        
        # Hidden layers
        self.hidden = nn.Sequential(
            nn.Linear(embedding_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.1)  # Add dropout for regularization
        )
        
        # Output layer to predict center character
        self.output = nn.Linear(hidden_dim, vocab_size)
        
    def forward(self, center_chars, context_chars):
        # Get embeddings for context characters
        context_embeds = self.embeddings(context_chars)  # [batch_size, context_size*2, embedding_dim]
        
        # Average context embeddings
        context_embeds = context_embeds.mean(dim=1)  # [batch_size, embedding_dim]
        context_embeds = self.batch_norm(context_embeds)
        
        # Pass through hidden layers
        hidden = self.hidden(context_embeds)  # [batch_size, hidden_dim]
        
        # Predict center character
        logits = self.output(hidden)  # [batch_size, vocab_size]
        
        return logits

In [ ]:
# Initialize model, optimizer, and loss function
model = CharacterEmbedding(vocab_size, embedding_dim)

# Use AdamW optimizer with weight decay for better regularization
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Keep using CrossEntropyLoss
criterion = nn.CrossEntropyLoss()

print(model)

In [ ]:
# Training loop
n_epochs = 50
losses, val_losses = [], []

for epoch in range(n_epochs):
    train_loss, val_loss = train_embedding_epoch( model, train_dataloader, validation_dataloader, optimizer, criterion )

    losses.append( train_loss )
    val_losses.append( val_loss )
    
    # Plot the losses
    plot_losses( losses, val_losses )

# Viz new embeddings

In [ ]:
# Get the trained embeddings (we use the main embeddings, not the context ones)
trained_embeddings = model.embeddings.weight.detach().numpy()

# Plot
plt.figure(figsize=(15, 10))
plt.scatter(trained_embeddings[:, 0], trained_embeddings[:, 1], alpha=0.5)

# Add character labels
for i, char in enumerate(chars):
    if char == '\n': char = 'newline'  # Make newline visible
    plt.annotate(char, (trained_embeddings[i, 0], trained_embeddings[i, 1]))

plt.title('Character Embeddings 2D')
plt.xlabel('First Component')
plt.ylabel('Second Component')
plt.grid(True)
plt.show()

# Now, train a model to produce new characters! 

## Creating Training Examples

Let's see how to create training examples for a language model. Each example will consist of a sequence of characters and the next character as the target.

In [ ]:
# Convert entire text to indices
data = torch.tensor([char_to_idx[ch] for ch in text], dtype=torch.long)

# Create sequences of context_length characters
context_length = 4
x = torch.stack([data[i:i+context_length] for i in range(len(data)-context_length)])
y = data[context_length:]

print(f'Shape of input sequences: {x.shape}')
print(f'Shape of target values: {y.shape}')

# Show an example
idx = 0  # First sequence
context = ''.join([idx_to_char[int(i)] for i in x[idx]])
next_char = idx_to_char[int(y[idx])]
print(f'\nExample:')
print(f'Context: "{context}"')
print(f'Next character: "{next_char}"')

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CharacterLM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, x):
        # x shape: (batch_size, sequence_length)
        embedded = self.embeddings(x)  # (batch_size, sequence_length, embedding_dim)
        lstm_out, _ = self.lstm(embedded)  # (batch_size, sequence_length, hidden_dim)
        logits = self.fc(lstm_out)  # (batch_size, sequence_length, vocab_size)
        return logits

# Model parameters
embedding_dim  = 16
hidden_dim     = 32
context_length = 16
batch_size     = 128

# Initialize model
model = CharacterLM(vocab_size, embedding_dim, hidden_dim)
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

print(f'Model parameters: {sum(p.numel() for p in model.parameters()):,}')

In [ ]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self, data, context_length):
        self.data = data
        self.context_length = context_length
    
    def __len__(self):
        return len(self.data) - self.context_length - 1
    
    def __getitem__(self, idx):
        # Get input sequence and target
        x = self.data[idx:idx + self.context_length]
        y = self.data[idx + 1:idx + self.context_length + 1]
        return x, y

# Create dataset and dataloader
dataset = TextDataset(data, context_length)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

print(f'Number of training examples: {len(dataset):,}')
print(f'Number of batches: {len(dataloader):,}')

In [ ]:
dataset[1]

In [ ]:
def train_epoch(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0
    
    for x, y in tqdm(dataloader, desc='Training'):
        # Forward pass
        logits = model(x)
        loss = criterion(logits.view(-1, vocab_size), y.view(-1))
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(dataloader)

In [ ]:
# Training parameters
n_epochs = 20

# Train the model
losses = []
for epoch in range(n_epochs):
    loss = train_epoch(model, dataloader, optimizer, criterion)
    losses.append(loss)

    # Plot the loss
    clear_output(wait=True)
    print(f'Epoch {epoch+1}/{n_epochs}, Loss: {loss:.4f}')

    plt.figure(figsize=(10, 5))
    plt.plot(losses, label='Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss Over Epochs')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
@torch.no_grad()
def generate_text(model, start_text, max_length=200, temperature=1.0):
    model.eval()
    
    # Convert start text to indices
    context = torch.tensor([char_to_idx[ch] for ch in start_text])
    generated_text = start_text
    
    for _ in range(max_length):
        # Get model predictions
        x = context[-context_length:].unsqueeze(0)  # Add batch dimension
        logits = model(x)
        probs = F.softmax(logits[0, -1] / temperature, dim=0)
        
        # Sample next character
        next_idx = torch.multinomial(probs, 1).item()
        next_char = idx_to_char[next_idx]
        
        # Update context and generated text
        generated_text += next_char
        context = torch.cat([context, torch.tensor([next_idx])])
    
    return generated_text

In [ ]:
# Generate some text
prompt = "To be, or not to be"

rich.print(f'\nPrompt: "{prompt}"')
rich.print('\n\nCompletions:')
rich.print(generate_text(model, prompt, max_length=200, temperature=0.3))

In [ ]:
# Generate some text
prompt = "Oh Romeo, Romeo, "

rich.print(f'\nPrompt: "{prompt}"')
rich.print('\n\nCompletions:')
rich.print(generate_text(model, prompt, max_length=200, temperature=0.5))

In [ ]:
# Create a Shakesperian name! 
# Since the learned representation contains uppercase for character names 
prompt = "MARC"

rich.print(f'\nPrompt: "{prompt}"')

completion = generate_text(model, prompt, max_length=10, temperature=0.5)
# rich.print(completion)
rich.print( f'Shakespearean name: "{completion.split(":")[0]}"' )